In [ ]:
pip install pygame 

In [7]:
import pygame
import sys

# --- ĐƯỜNG DẪN ASSET ---
ASSET_PATH = "C:/Users/ADMIN/Downloads/pygame_assets/"

# --- THIẾT LẬP GAME CƠ BẢN ---
pygame.init()
SCREEN_WIDTH = 960
SCREEN_HEIGHT = 480
SCREEN = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Level 3: WaterWar Adventure")
CLOCK = pygame.time.Clock()
FPS = 60

GRAVITY = 0.32
PLAYER_JUMP_VEL = 9
PLAYER_SPEED = 3
BLOCK_SIZE = 48

# --- MÀU ---
WHITE = (255, 255, 255)
BLUE = (0, 0, 255)
LIGHT_BLUE = (173, 216, 230)
RED = (255, 0, 0)
DARK_GRAY = (50, 50, 50)

# --- HÀM TẢI ẢNH ---
def load_image(name, scale=None):
    img = pygame.image.load(ASSET_PATH + name).convert_alpha()
    if scale:
        img = pygame.transform.scale(img, scale)
    return img

# --- ASSETS ---
BG_IMG = load_image("level3bg.png", (SCREEN_WIDTH, SCREEN_HEIGHT))
PLAYER_IDLE_IMG = load_image("idle-effect-sheet.png", (75, 70))
PLAYER_RUN_IMG = load_image("running-effect-sheet.png", (60, 70))
PLAYER_JUMP_IMG = load_image("jumping-effect-sheet.png", (60, 70))
CHECKPOINT_IMG = load_image("checkpoint.png", (160, 190))
TRIGGER_IMG = load_image("unactivated_typhoon_trap.png", (100, 80))
WATER_IMG = load_image("water1.png",(SCREEN_WIDTH, SCREEN_HEIGHT))
BLOCK_IMG = load_image("block.png",(48,48))
BLOCK_IMG = pygame.transform.scale(BLOCK_IMG, (BLOCK_SIZE, BLOCK_SIZE))
FREEZE_IMG = load_image("freeze.jpg",(SCREEN_WIDTH, SCREEN_HEIGHT))
PANE_IMG = load_image("freeze_pane.1.png",(SCREEN_WIDTH, SCREEN_HEIGHT))

#============ CLASS PLAYER ================

class Player(pygame.sprite.Sprite):
    def __init__(self, x, y):
        super().__init__()
        self.width, self.height = 60,70
        self.image = PLAYER_IDLE_IMG
        self.rect = self.image.get_rect(topleft=(x, y))
        self.vel_x = 0
        self.vel_y = 0
        self.on_ground = False
        self.is_alive = True
        self.is_swimming = False
        self.can_be_frozen = True
        self.state = "idle"  # idle, run, jump

    def move(self, keys):
        if not self.is_alive:
            return
        self.vel_x = 0

        if keys[pygame.K_LEFT]:
            self.vel_x = -PLAYER_SPEED
            self.state = "run"
        elif keys[pygame.K_RIGHT]:
            self.vel_x = PLAYER_SPEED
            self.state = "run"
        else:
            self.state = "idle"

        if keys[pygame.K_SPACE] or keys[pygame.K_UP]:
            if self.is_swimming:
                self.vel_y = -3
                self.state = "jump"
            elif self.on_ground:
                self.vel_y = -PLAYER_JUMP_VEL
                self.on_ground = False
                self.state = "jump"

    def apply_freeze_death(self):
        if self.is_alive and self.can_be_frozen:
            self.is_alive = False
            self.image = pygame.Surface((self.width, self.height))
            self.image = pygame.transform.scale(PANE_IMG, (self.width*3.5, self.height*3))
            self.vel_x = 0
            self.vel_y = 0

    def set_frozen_solid(self, is_solid):
        if is_solid:
            self.vel_x = 0
            self.vel_y = 0
            self.on_ground = True

    def update(self, platforms):
        if not self.is_alive:
            return

        # Trọng lực
        if self.is_swimming:
            self.vel_y += 0.05
            if self.vel_y > 3:
                self.vel_y = 3
        elif not self.on_ground:
            self.vel_y += GRAVITY

        # Cập nhật vị trí ngang
        self.rect.x += self.vel_x
        for platform in platforms:
            if self.rect.colliderect(platform):
                if self.vel_x > 0:
                    self.rect.right = platform.left
                elif self.vel_x < 0:
                    self.rect.left = platform.right
                self.vel_x = 0
        self.on_ground = False

        # Cập nhật vị trí dọc
        self.rect.y += self.vel_y
        for platform in platforms:
            if self.rect.colliderect(platform):
                if self.vel_y > 0:
                    self.rect.bottom = platform.top
                    self.vel_y = 0
                    self.on_ground = True
                    self.is_swimming = False
                elif self.vel_y < 0:
                    self.rect.top = platform.bottom
                    self.vel_y = 0

        # Cập nhật hình theo state
        if self.state == "idle":
            self.image = PLAYER_IDLE_IMG
        elif self.state == "run":
            self.image = PLAYER_RUN_IMG
        elif self.state == "jump":
            self.image = PLAYER_JUMP_IMG

        # Giới hạn màn hình
        if self.rect.left < 0:
            self.rect.left = 0
        if self.rect.right > SCREEN_WIDTH:
            self.rect.right = SCREEN_WIDTH
        if self.rect.top < 0:
            self.rect.top = 0

        if self.rect.top > SCREEN_HEIGHT:
            self.is_alive = False

#----------WATER-----------
class Water(pygame.sprite.Sprite):
    def __init__(self, x, y, width, height, freeze_speed=0.5):
        super().__init__()
        self.full_rect = pygame.Rect(x, y, width, height)
        self.current_rect = pygame.Rect(x, y + height, width, 0)
        self.freeze_speed = freeze_speed
        self.current_height_float = 0.0
        self.is_frozen = False
        self.is_fully_frozen = False
        self.surface_y = y

        self.normal_image = WATER_IMG
        self.normal_image = pygame.transform.scale(WATER_IMG, (width, height))
        self.frozen_image = FREEZE_IMG
        self.frozen_image = pygame.transform.scale(FREEZE_IMG, (width, height))
        self.image = self.normal_image
        self.rect = self.full_rect

    def start_freezing(self):
        if not self.is_frozen and not self.is_fully_frozen:
            self.is_frozen = True
            print("Bắt đầu đóng băng nước...")

    def update_freezing(self):
        if not self.is_frozen or self.is_fully_frozen:
            return

        self.current_height_float += self.freeze_speed
        new_height = int(self.current_height_float)
        if new_height > self.full_rect.height:
            new_height = self.full_rect.height

        if new_height != self.current_rect.height:
            self.current_rect.height = new_height
            self.current_rect.y = self.full_rect.bottom - new_height
            self.image = self.normal_image.copy()
            ice_rect_on_image = pygame.Rect(
                0, self.full_rect.height - new_height, self.full_rect.width, new_height
            )
            self.image.blit(self.frozen_image, ice_rect_on_image.topleft, area=ice_rect_on_image)

        if new_height >= self.full_rect.height:
            self.is_fully_frozen = True
            self.is_frozen = False
            self.current_rect.y = self.surface_y
            self.current_rect.height = self.full_rect.height
            self.image = self.frozen_image.copy()

    def draw(self, surface):
        surface.blit(self.image, self.rect)

#-------ICE TRIGGER---------
class IceActiveTrigger(pygame.sprite.Sprite):
    def __init__(self, x, y, width, height, sink_speed=1):
        super().__init__()
        self.image = TRIGGER_IMG
        self.rect = self.image.get_rect(topleft=(SCREEN_WIDTH - 480, SCREEN_HEIGHT - 20))
        self.original_y = y
        self.is_activated = False
        self.is_sinking = False
        self.sink_speed = sink_speed
        self.bottom_y = y + 150

    def activate(self):
        if not self.is_activated:
            self.is_activated = True
            self.is_sinking = True
            print("Vật thể kích hoạt bắt đầu chìm.")

    def update(self):
        if self.is_sinking:
            if self.rect.y < self.bottom_y:
                self.rect.y += self.sink_speed
            else:
                self.is_sinking = False

#---------CHECK POINT-----------
class Checkpoint(pygame.sprite.Sprite):
    def __init__(self, x, y):
        super().__init__()
        self.image = CHECKPOINT_IMG
        self.rect = self.image.get_rect(topleft=(x, y))
        self.is_active = True

def check_collider_and_freeze(player, trigger, water):
    if not trigger.is_activated and player.rect.colliderect(trigger.rect):
        if player.is_swimming:
            trigger.activate()
            water.start_freezing()


def check_water_collision(player, water):
    if player.rect.colliderect(water.full_rect) and not water.is_fully_frozen:
        player.is_swimming = True
    elif player.is_swimming and not player.rect.colliderect(water.full_rect):
        player.is_swimming = False
        player.on_ground = False
    player.can_be_frozen = player.is_swimming


def update_freeze_mechanics(player, water, trigger):
    trigger.update()
    if water.is_frozen:
        water.update_freezing()

    if player.is_swimming and player.can_be_frozen and player.rect.colliderect(water.current_rect):
        player.apply_freeze_death()

    if water.is_fully_frozen and player.rect.colliderect(water.full_rect):
        if player.rect.bottom > water.surface_y and player.vel_y >= 0:
            player.rect.bottom = water.surface_y
            player.set_frozen_solid(True)
            player.is_swimming = False


def check_win(player, checkpoint):
    if player.is_alive and checkpoint.is_active and player.rect.colliderect(checkpoint.rect):
        print("--- WIN ---")
        checkpoint.is_active = False
        return True
    return False

def draw_bank(surface, rect):
    width_blocks = rect.width // BLOCK_SIZE
    height_blocks = rect.height // BLOCK_SIZE
    for i in range(width_blocks):
        for j in range(height_blocks):
            x = rect.x + i * BLOCK_SIZE
            y = rect.y + j * BLOCK_SIZE
            surface.blit(BLOCK_IMG, (x, y))

def run_game():
    left_bank = pygame.Rect(0, SCREEN_HEIGHT - BLOCK_SIZE * 2, BLOCK_SIZE * 6, BLOCK_SIZE * 2)
    right_bank = pygame.Rect(SCREEN_WIDTH - BLOCK_SIZE * 4, SCREEN_HEIGHT - BLOCK_SIZE * 2, BLOCK_SIZE * 4, BLOCK_SIZE * 2)

    top_border = pygame.Rect(0, 0, SCREEN_WIDTH, BLOCK_SIZE)  # viền ngang trên
    left_border = pygame.Rect(0, 0, BLOCK_SIZE, SCREEN_HEIGHT - BLOCK_SIZE * 2)  # viền dọc trái
    right_border = pygame.Rect(SCREEN_WIDTH - BLOCK_SIZE, 0, BLOCK_SIZE, SCREEN_HEIGHT - BLOCK_SIZE * 2)  # viền dọc phải

    water_width = SCREEN_WIDTH - left_bank.width - right_bank.width
    water = Water(left_bank.right, SCREEN_HEIGHT - BLOCK_SIZE * 2, water_width, BLOCK_SIZE * 2, freeze_speed=0.5)

    trigger = IceActiveTrigger(water.full_rect.centerx, SCREEN_HEIGHT - 30, 100, 100)
    checkpoint = Checkpoint(SCREEN_WIDTH - 200, right_bank.top - 130)
    player = Player(50, left_bank.top - 60)
    has_won = False
    platforms = [left_bank, right_bank]

    while True:
        for e in pygame.event.get():
            if e.type == pygame.QUIT:
                pygame.quit()
                sys.exit()

        keys = pygame.key.get_pressed()
        player.move(keys)

        check_collider_and_freeze(player, trigger, water)
        check_water_collision(player, water)
        update_freeze_mechanics(player, water, trigger)
        player.update(platforms)

        if check_win(player, checkpoint):
            has_won = True

        # --- Vẽ ---
        SCREEN.blit(BG_IMG, (0,0))
        draw_bank(SCREEN, top_border)
        draw_bank(SCREEN, left_border)
        draw_bank(SCREEN, right_border)
        draw_bank(SCREEN, left_bank)
        draw_bank(SCREEN, right_bank)
        water.draw(SCREEN)
        SCREEN.blit(trigger.image, trigger.rect)
        SCREEN.blit(checkpoint.image, checkpoint.rect)
        SCREEN.blit(player.image, player.rect)
        font = pygame.font.Font("D:\Python NEU\PixelifySans-VariableFont_wght.ttf", 35)
        SCREEN.blit(font.render("LET'S GO PENGUIN !", False, 'yellow'), (320,60))

        if has_won:
            font = pygame.font.Font("D:\Python NEU\PixelifySans-VariableFont_wght.ttf", 74)
            SCREEN.blit(font.render("VICTORY!", True, BLUE ), (330, 120))
            font1 = pygame.font.Font("D:\Python NEU\PixelifySans-VariableFont_wght.ttf", 20)
            SCREEN.blit(font1.render("Penguin has reached to the snowball!", True, "gold" ), (300, 200))
            
        elif not player.is_alive:
            font = pygame.font.Font("C:/Users/ADMIN/Downloads/game-paused-demo/Game Paused DEMO.otf", 74)
            text_surface = font.render("GAME OVER :( !!", True, RED)
            SCREEN.blit(text_surface, (300, 200))
            

        pygame.display.flip()
        CLOCK.tick(FPS)


if __name__ == "__main__":
    run_game()


<>:333: SyntaxWarning: invalid escape sequence '\P'
<>:337: SyntaxWarning: invalid escape sequence '\P'
<>:339: SyntaxWarning: invalid escape sequence '\P'
<>:333: SyntaxWarning: invalid escape sequence '\P'
<>:337: SyntaxWarning: invalid escape sequence '\P'
<>:339: SyntaxWarning: invalid escape sequence '\P'
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_14984\688894497.py:333: SyntaxWarning: invalid escape sequence '\P'
  font = pygame.font.Font("D:\Python NEU\PixelifySans-VariableFont_wght.ttf", 35)
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_14984\688894497.py:337: SyntaxWarning: invalid escape sequence '\P'
  font = pygame.font.Font("D:\Python NEU\PixelifySans-VariableFont_wght.ttf", 74)
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_14984\688894497.py:339: SyntaxWarning: invalid escape sequence '\P'
  font1 = pygame.font.Font("D:\Python NEU\PixelifySans-VariableFont_wght.ttf", 20)


--- WIN ---
Vật thể kích hoạt bắt đầu chìm.
Bắt đầu đóng băng nước...


SystemExit: 

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_14984\688894497.py:333: SyntaxWarning: invalid escape sequence '\P'
  font = pygame.font.Font("D:\Python NEU\PixelifySans-VariableFont_wght.ttf", 35)
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_14984\688894497.py:337: SyntaxWarning: invalid escape sequence '\P'
  font = pygame.font.Font("D:\Python NEU\PixelifySans-VariableFont_wght.ttf", 74)
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_14984\688894497.py:339: SyntaxWarning: invalid escape sequence '\P'
  font1 = pygame.font.Font("D:\Python NEU\PixelifySans-VariableFont_wght.ttf", 20)
